# Download Data

In [2]:
import requests
import pandas as pd
from io import BytesIO

def download_csv(csv_url):
    response = requests.get(csv_url)
    response.raise_for_status()

    # Qui puoi modificare 'sep' o aggiungere altri parametri se necessario
    try:
        df = pd.read_csv(BytesIO(response.content), sep=',')
    except pd.errors.ParserError:
        df = pd.read_csv(BytesIO(response.content), sep=';', error_bad_lines=False)

    return df

In [3]:
recipes_df = download_csv("https://media.githubusercontent.com/media/DonatoFe11/SustainaMeal_Case_Study/master/data/final_recipes_set.csv")

In [4]:
recipes_df

,title,sustainability_score,sustainability_label,Unnamed: 0,recipe_id,description,author_id,duration,directions,ingredients,...,protein [g],direction_size,ingredients_sizes,who_score,fsa_score,nutri_score,normalization_comment,ingredient_food_kg_urls,ingredient_food_kg_names,healthiness_label
0,Boiled Radishes,0.002502,0,294268,179840,These taste a lot like new potatoes. The boil...,254469,22.0,"['Trim ends and bad spots off Radishes.', 'Boi...","['radishes size bag bunch', 'water radishes']",...,0.8,2,2,0.293040,1.000,0.75,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['applesauce', 'garlic clove', 'ginger', 'mola...",0
1,Apple Cider Reduction,0.002623,0,59722,521756,I saw a post about this on Facebook and gave i...,171084,155.0,"['In 6-8 quart stock pan, measure out 2 cups o...","['gallon apple cider fresh', 'gallon apple cid...",...,0.0,10,2,0.214286,0.750,0.75,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['bean sprouts', 'chicken thighs', 'fresh ging...",1
2,Homemade Apple Pectin Stock,0.002623,0,159910,25188,Easy to make and works well.,10404,25.0,['Slice unpeeled apples including cores and se...,"['tart apple', 'water pound apples']",...,1.2,11,2,0.278414,0.750,0.75,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['baking powder', 'baking soda', 'cinnamon', '...",0
3,Easy Applesauce,0.002623,0,421226,264179,Use fresh cooking apples for this for best res...,37636,50.0,['Place apples and cider into a large saucepan...,"['apples', 'apple cider']",...,0.5,3,2,0.276525,0.875,0.75,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['carrots', 'fresh peas', 'fresh tomatoes', 'g...",0
4,Cider Baked Sausage,0.002752,0,374088,101966,A very easy to make main dish perfect for fall...,30534,50.0,"['Preheat oven to 350°.', 'Prick sausages with...","['breakfast sausage links', 'apples Smith', 'o...",...,46.4,4,4,0.146714,0.125,0.00,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['Shedd Spread Country Crock', 'cayenne pepper...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93743,Curried Lamb on Rice,0.951024,2,103145,373244,"With modification this recipe is from ""The Dia...",133174,65.0,"['Over medium heat, melt the butter in a large...","['lamb', 'medium onion', 'cloves', 'butter', '...",...,28.7,5,13,0.223549,0.625,0.25,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['Dijon mustard', 'Polish sausage', 'apricot p...",1
93744,Rich Lamb Curry,0.960504,2,442694,268247,The recipe isn't as time consuming as it looks...,491437,110.0,"['Heat 1 tbsp olive oil in dutch oven. ', 'Co...","['oil', 'lamb', 'flour', 'salt pepper', 'onion...",...,36.1,9,21,0.153328,0.000,0.00,NaN,NaN,NaN,1
93745,"Middle Eastern Slow-Cooked Stew With Lamb, Chi...",0.968458,2,359960,144850,From Cooking Light. Per 3/4 c. serving: 310 ca...,37779,112.0,['Let the oil get heating in a large pot over ...,"['oil', 'lamb', 'onions rings', 'water', 'clov...",...,20.8,12,19,0.205566,0.250,0.25,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['anchovies', 'celery ribs', 'diced tomatoes',...",1
93746,Five Meat Chili Con Carne With Beans,0.971454,2,37637,13568,yum,20571,210.0,"['In a large pot brown meat in stages, about a...","['beef', 'pork', 'lamb', 'sausage', 'sausage',...",...,14.5,7,15,0.197568,0.250,0.25,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"[""French\\'s French fried onions"", 'crabmeat',...",1


# SustainaMeal

Initializes the system by loading the data and preparing the embeddings.

In [5]:
from HeASe.sustainameal import SustainaMeal

sm = SustainaMeal(
    recipes_df=recipes_df,
    nutrients=['calories [cal]', 'totalFat [g]', 'saturatedFat [g]', 'cholesterol [mg]', 'sodium [mg]', 'dietaryFiber [g]', 'sugars [g]', 'protein [g]'],
    transformer_name='davanstrien/autotrain-recipes-2451975973'
)

/home/festa/SustainaMeal_Case_Study/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Processing Titles embeddings: 100%|██████████| 93748/93748 [18:40<00:00, 83.69batch/s]


## Find Similar Recipes by Title

Debug function which return the list of recipes titles similar to the given input text (STEP 1).

        
        """
        Parameters

        :param input_text: The input text to find similar recipes for.
        :param k: Number of similar recipes to return.
        
        :return: A list of tuples with similar recipes and their similarity scores.
        """

In [6]:
similar_by_title = sm.get_similar_by_title('Creamy Lemon Asparagus Risotto', 10)

Processing Titles embeddings: 100%|██████████| 1/1 [00:00<00:00, 48.31batch/s]

In [7]:
df = pd.DataFrame(similar_by_title, columns=['Title', 'Similarity'])
df

,Title,Similarity
0,"(171027, Creamy Lemon Asparagus Risotto)",1.000000
1,"(153003, Creamy Spinach Risotto)",0.959704
2,"(237255, Creamy Lemon Fettuccine)",0.957575
3,"(167231, Creamy Apple Raisin Oatmeal)",0.957208
4,"(394589, Creamy Elegant Rice Casserole)",0.952827
5,"(218448, Creamy Blueberry Gelatin Salad)",0.951957
6,"(152709, Creamy Asparagus Meatloaf)",0.948904
7,"(249025, Creamy Cornbread Casserole)",0.948449
8,"(204569, Creamy Italian Pasta Salad)",0.946278
9,"(19990, Creamy Broccoli Salad)",0.945645


## Find Similar Recipes by tags and Title

Finds recipes similar (nutrients similarity) to the given input text. (STEP 1-2).
        
        """
        Parameters

        :param input_text: The input text to find similar recipes for.
        :param k: Number of similar recipes to return.
        :param acceptable_tags: List of tags considered acceptable for filtering recipes.
        :param match_all_tags: Matching strategy
        
        :return: A list of tuples with similar recipes and their similarity scores.
        """

In [8]:
sr = sm.find_similar_recipes('Creamy Lemon Asparagus Risotto', 10,
                        acceptable_tags=['appetizers', 'main-dish', 'side-dishes', 'fruits', 'desserts',
                                          'breakfast', 'pasta-rice-and-grains', 'beverages', 'drinks'],
                        match_all_tags=True)

Processing Titles embeddings: 100%|██████████| 1/1 [00:00<00:00, 47.43batch/s]

Tags to match: ['side-dishes', 'pasta-rice-and-grains']
Number of filtered recipes: 2610


In [16]:
df = pd.DataFrame(sr)
df

,recipe_id,title,similarity
0,263276,"Rice Cooker "" Risotto"" With Carrots and Onions",0.998882
1,62907,Super Easy Pasta Salad,0.996196
2,17270,Quick and Easy Saffron Rice,0.995591
3,162565,Rice With Almonds & Raisins,0.995430
4,175027,Rice Cooker Rice Pilaf,0.995328
5,365442,Orange-Cinnamon Rice Pilaf,0.995262
6,381356,Green Coriander Rice,0.995079
7,334876,Italian Inspired Microwave Rice,0.995014
8,235297,Authentic Mexican - Arroz Roja,0.994981
9,52485,Garlicky Rice,0.994838


## Order results by healthiness score

  Order the recipes obtained previously.
        
        """

        :param (optional) nearest_recipes: Dataframe to order, if none the dataframe computed by find_similar_recipes will be used
        :param score (default:'who_score') : The column name used as the primary sorting criterion.

        :return: A DataFrame of recipes ordered by the specified score.

        """

In [10]:
sm.order_recipe_by_healthiness()

,title,similarity_score,who_score,healthiness_increment
21800,Rice Cooker Rice Pilaf,0.995328,0.248747,10.283248
53119,Quick and Easy Saffron Rice,0.995591,0.234049,3.766948
19758,Authentic Mexican - Arroz Roja,0.994981,0.232525,3.091250
5264,Super Easy Pasta Salad,0.996196,0.230867,2.356268
21852,Garlicky Rice,0.994838,0.230072,2.003852
32626,Rice With Almonds & Raisins,0.995430,0.220635,-2.180333
71284,Italian Inspired Microwave Rice,0.995014,0.218376,-3.181688
29545,Orange-Cinnamon Rice Pilaf,0.995262,0.217653,-3.502509
29818,"Rice Cooker "" Risotto"" With Carrots and Onions",0.998882,0.215539,-4.439704
56194,Green Coriander Rice,0.995079,0.215489,-4.461767


## Order results by sustainability score

  Order the recipes obtained previously.
        
        """

        :param (optional) nearest_recipes: Dataframe to order, if none the dataframe computed by find_similar_recipes will be used
        :param score (default:'sustainability_label') : The column name used as the primary sorting criterion.
        :param secondary_sort_field (default:'who_score'): The column name used as the secondary sorting criterion.

        :return: A Dataframe with recipes ordered by the given metric.

        """

In [11]:
sm.order_recipe_by_sustainability()

,title,similarity_score,sustainability_score,who_score,sustainability_increment
5264,Super Easy Pasta Salad,0.996196,0.948860,0.230867,35.142954
19758,Authentic Mexican - Arroz Roja,0.994981,0.893255,0.232525,27.223358
21800,Rice Cooker Rice Pilaf,0.995328,0.890097,0.248747,26.773572
21852,Garlicky Rice,0.994838,0.890019,0.230072,26.762385
29545,Orange-Cinnamon Rice Pilaf,0.995262,0.874427,0.217653,24.541727
29818,"Rice Cooker "" Risotto"" With Carrots and Onions",0.998882,0.873149,0.215539,24.359661
32626,Rice With Almonds & Raisins,0.995430,0.866283,0.220635,23.381840
53119,Quick and Easy Saffron Rice,0.995591,0.753798,0.234049,7.360874
56194,Green Coriander Rice,0.995079,0.725922,0.215489,3.390612
71284,Italian Inspired Microwave Rice,0.995014,0.691571,0.218376,-1.501808
